In [1]:
%pylab inline
import jax
import jax.numpy as jnp
import tensorflow_probability as tfp; tfp = tfp.substrates.jax
tfd = tfp.distributions
tfb = tfp.bijectors

from jaxlie import SO3
from so3dm.distributions.isotropic_gaussian import IsotropicGaussianSO3
from so3dm.plotting import visualize_so3_probabilities, visualize_so3_density

import tensorflow_datasets as tfds
import tensorflow as tf

from flax.metrics import tensorboard
import haiku as hk
import optax

from tqdm import tqdm

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


2023-05-18 14:22:57.613499: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/packages/AI/tensorflow_23.02-2.10.0-py3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.neighbors import radius_neighbors_graph
import jraph
from jraph import GraphConvolution
from jraph._src import utils as jraph_utils
from jraph._src import graph as gn_graph

import networkx as nx
import jax.tree_util as tree
from typing import Any, Callable, Dict, List, Optional, Tuple

In [3]:
import os
os.environ["XLA_FLAGS"]="--xla_gpu_force_compilation_parallelism=1"

In [4]:
%pylab inline
%load_ext autoreload
%autoreload 2
import os
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/tmp/ipykernel_75360/1757885639.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
jnp.linalg.inv(jnp.eye(3))

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [6]:
rng = hk.PRNGSequence(42)


In [7]:
import pickle
from halotools_ia.correlation_functions import  ed_3d, ee_3d,gi_plus_3d, gi_plus_projected, ii_minus_3d, ii_minus_projected, ii_plus_3d, ii_plus_projected, ed_projected, ed_3d_one_two_halo_decomp


In [8]:
tng = pickle.load(  open('/jet/home/yjagvara/SO3Diffusion_Tidal/TNG100-1_99_non-reduced_galaxy_shapes_multi_scale_1024_MLP_only_cent.pkl', "rb" ) )
tng = tng[tng['dm_mass']>0]
tng = tng[log10(tng['dm_mass']*10**10)>9]
tng = tng[log10(tng['mass']*10**10)>9]

tng['mass'] = log10(tng['mass']*10**10)
tng['mass'] = (tng['mass']  -jnp.mean(tng['mass']))/ jnp.std(tng['mass']) 
tng['dm_mass'] = log10(tng['dm_mass']*10**10)
tng['dm_mass'] = (tng['dm_mass']  -jnp.mean(tng['dm_mass']))/ jnp.std(tng['dm_mass']) 

#tng = tng[tng['central_bool']==1.0]


## Process the data for SO(3)

In [9]:
flat_mat_I = np.array( (tng['dm_av_x'], tng['dm_av_y'], tng['dm_av_z'], 
tng['dm_bv_x'], tng['dm_bv_y'], tng['dm_bv_z'],
tng['dm_cv_x'], tng['dm_cv_y'], tng['dm_cv_z'])).T

rot_mat_I = np.reshape(flat_mat_I, (len(flat_mat_I), 3, 3), order='F' )

for i in range (len(rot_mat_I)):
    if np.linalg.det(rot_mat_I[i] ) < 0:
        rot_mat_I[i] = rot_mat_I[i] @ (np.eye(3)*-1)
        
flat_mat_T = np.array( (   tng['tid_av_x_0.5_1024'], tng['tid_av_y_0.5_1024'], tng['tid_av_z_0.5_1024'], 
tng['tid_bv_x_0.5_1024'], tng['tid_bv_y_0.5_1024'], tng['tid_bv_z_0.5_1024'],
tng['tid_cv_x_0.5_1024'], tng['tid_cv_y_0.5_1024'], tng['tid_cv_z_0.5_1024']   )).T


flat_scalar_T =  np.array((tng['tid_a_0.5_1024'],  tng['tid_b_0.5_1024'],  tng['tid_c_0.5_1024'])).T
rot_mat_T = np.reshape(flat_mat_T, (len(flat_mat_T), 3, 3), order='F' )

for i in range (len(rot_mat_T)):
    if np.linalg.det(rot_mat_T[i] ) < 0:
        rot_mat_T[i] = rot_mat_T[i] @ (np.eye(3)*-1)
        

pos_tng = np.array([tng['gal_pos_x'], tng['gal_pos_y'], tng['gal_pos_z']]).T
quat_I = jax.vmap(lambda x: SO3.from_matrix(x).wxyz) (rot_mat_I)
quat_T = jax.vmap(lambda x: SO3.from_matrix(x).wxyz) (rot_mat_T)
ax_ang_T = jax.vmap(lambda x: SO3(x).log() ) (quat_T)
data_dict = {'quat_I': quat_I, 'quat_T': quat_T, 'scalar_T': flat_scalar_T, 'a': jnp.array(tng['dm_a']), 'b': jnp.array(tng['dm_b']), 'c': jnp.array(tng['dm_c']) }



In [10]:
noise_dist_std = 2

## Let's build the graphs

In [11]:
group_key='GroupID'
pos_key=['gal_pos_x', 'gal_pos_y', 'gal_pos_z']
scalar_key = ['mass', 'dm_mass']
catalog = tng

 
# It takes a minute but we precompute all the graphs and data
# Identify the individual groups and pre-extract the relevant data

group_ids = catalog[group_key].astype(jnp.int32)
gids, idx = jnp.unique(group_ids, return_index=True) # gids are the unique group ids, in other words All the host halo IDS uniquely  extracted     idx = The indices of the first occurrences of the unique values in the original array. i.e index of the central galaxy
Position = jnp.array(catalog[pos_key].to_pandas())
Scalars = jnp.array(catalog[scalar_key].to_pandas())
quat_I = jnp.array(quat_I)
quat_T = jnp.array(quat_T)
flat_scalar_T = jnp.array(flat_scalar_T)

Scalars = jnp.concatenate([Scalars, flat_scalar_T], axis=-1)
quat_I_T = jnp.concatenate([quat_I, quat_T], axis=-1)

In [12]:
Position.shape

(17457, 3)

In [13]:
n_scalar = 5 #number of scalar features

In [14]:
graph_radius = 1 #Mpc/h
graphs_list = []
node_features_list = []
positions_list = []
direction_to_COM_list = []
quat_list = []


for gid in tqdm(gids):
    
    g = np.where(group_ids == gid)[0]
    Positions_for_group = Position[g]
    Features_for_group = Scalars[g]
    quat_I_T_for_group = quat_I_T[g]

    #keep only halos with more than 10 galaxies
    if Features_for_group.shape[0] >10:

        # Compute adjacency matrix for each entry
        graph = radius_neighbors_graph(Positions_for_group, graph_radius, mode='connectivity',
                                   include_self=False)

        Positions_for_group_COM =  (Positions_for_group - Positions_for_group[0])
        directions_to_COM = Positions_for_group_COM/jnp.linalg.norm(Positions_for_group_COM, axis=-1)[..., jnp.newaxis]


        directions_to_COM = directions_to_COM.at[jnp.isnan(directions_to_COM)].set(0.0)

        graphs_list.append(graph)
        node_features_list.append(Features_for_group)

        direction_to_COM_list.append(directions_to_COM)
        positions_list.append(Positions_for_group_COM)
        
        quat_list.append(quat_I_T_for_group)
        
        

100%|██████████| 10292/10292 [00:41<00:00, 245.43it/s]


In [15]:
len(graphs_list)

127

In [16]:
max_n_node = graphs_list[0].tocoo().shape[0]
max_n_edge = graphs_list[0].tocoo().nnz


In [17]:
senders_list = []
receivers_list = []
n_node_list = []
n_edge_list = []

In [18]:
for single_graph in tqdm(graphs_list):
    single_graph =  single_graph.tocoo()
    
 
    senders = single_graph.row.astype(jnp.int64)
    receivers = single_graph.col.astype(jnp.int64)
    values = single_graph.data.astype(jnp.int64)
    n_node = jnp.asarray([single_graph.shape[0]])
 
    values = np.array([0]) if values.size == 0 else values
    n_edge = np.array([np.sum(values)])
    senders = np.repeat(senders, values)
    receivers = np.repeat(receivers, values)
    
    if len(senders) < max_n_edge:
        senders = jnp.concatenate( [senders, -1*jnp.ones( max_n_edge - len(senders)  )] )
        receivers = jnp.concatenate( [receivers, -1*jnp.ones(max_n_edge - len(receivers)   )] )
        
    
     
    senders_list.append( jnp.asarray(senders).astype(jnp.int32)  )
    receivers_list.append( jnp.asarray(receivers).astype(jnp.int32) )
    
    n_node_list.append( jnp.asarray(n_node).astype(jnp.int32) )
    n_edge_list.append( jnp.asarray(n_edge).astype(jnp.int32) )
    
    
    
    

100%|██████████| 127/127 [00:05<00:00, 23.32it/s]


In [19]:
node_features_list_padded = []
positions_list_padded = []
direction_to_COM_list_padded = []
quat_list_padded = []

In [20]:
#here padding the node features with np.inf
for quat in tqdm(quat_list ):
    if len(quat) < max_n_node:
        
        quat = jnp.concatenate( [quat, 0*jnp.ones( (max_n_node-len(quat), jnp.squeeze(quat ).shape[-1])  )] )
        
    quat_list_padded.append( jnp.asarray(quat).astype(jnp.float32)  )
        

100%|██████████| 127/127 [00:01<00:00, 68.56it/s]


In [21]:
#here padding the node features with np.inf
for direction_to_COM in tqdm(direction_to_COM_list ):
    if len(direction_to_COM) < max_n_node:
        
        direction_to_COM = jnp.concatenate( [direction_to_COM, 0*jnp.ones( (max_n_node-len(direction_to_COM), jnp.squeeze(direction_to_COM ).shape[-1])  )] )
        
    direction_to_COM_list_padded.append( jnp.asarray(direction_to_COM).astype(jnp.float32)  )
        

100%|██████████| 127/127 [00:01<00:00, 67.82it/s]


In [22]:
#here padding the node features with np.inf
for node_features in tqdm(node_features_list ):
    if len(node_features) < max_n_node:
        
        node_features = jnp.concatenate( [node_features, 0*jnp.ones( (max_n_node-len(node_features), jnp.squeeze(node_features ).shape[-1])  )] )
        
    node_features_list_padded.append( jnp.asarray(node_features).astype(jnp.float32)  )
        

100%|██████████| 127/127 [00:01<00:00, 67.94it/s]


In [23]:
#here padding the node features with np.inf
for positions in tqdm(positions_list ):
    if len(positions) < max_n_node:
        
        positions = jnp.concatenate( [positions, 0*jnp.ones( (max_n_node-len(positions), jnp.squeeze(positions ).shape[-1])  )] )
        
    positions_list_padded.append( jnp.asarray(positions).astype(jnp.float32)  )
        

100%|██████████| 127/127 [00:00<00:00, 1362.45it/s]


In [24]:
senders_list = jnp.array(senders_list)
receivers_list = jnp.array(receivers_list)

n_node_list = jnp.array(n_node_list)
n_edge_list = jnp.array(n_edge_list)

node_features_list_padded = jnp.array(node_features_list_padded)
positions_list_padded = jnp.array(positions_list_padded)
direction_to_COM_list_padded = jnp.array(direction_to_COM_list_padded)
quat_list_padded = jnp.array(positions_list_padded)

In [25]:
concat_feats = jnp.concatenate([node_features_list_padded, quat_list_padded,  direction_to_COM_list_padded, positions_list_padded], axis=-1)

In [26]:
GP =  gn_graph.GraphsTuple(
      nodes=concat_feats,  
      edges=-1*jnp.ones_like(n_node_list),
      receivers= receivers_list   ,
      senders= senders_list ,
      globals=  -1*jnp.ones_like(n_node_list),
      n_node=n_node_list,
      n_edge=n_edge_list)

In [27]:
batch_size = 64

dset = tf.data.Dataset.from_tensor_slices(GP)

dset = dset.repeat()
dset = dset.shuffle(buffer_size=10000)
dset = dset.batch(batch_size)
dset = dset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
dset = dset.as_numpy_iterator()
_ = next(dset)

In [28]:
g = next(dset)

In [29]:


dset2 = tf.data.Dataset.from_tensor_slices(GP)

dset2 = dset2.repeat()
dset2 = dset2.shuffle(buffer_size=10000)
dset2 = dset2.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
dset2 = dset2.as_numpy_iterator()
g_init = next(dset2)

In [30]:
def f_e(feats: jnp.ndarray, in_dim) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.nets.MLP([in_dim , in_dim ], activation=jax.nn.silu)(feats)
  return net

def f_inf(m_ij: jnp.ndarray, in_dim) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.nets.MLP( [1,], activation=jax.nn.sigmoid)(m_ij)
  return net

def f_h(feats: jnp.ndarray, in_dim) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.nets.MLP([in_dim,  ], activation=jax.nn.silu)(feats)
  net = hk.Linear(in_dim)(net)
  return net


def f_x(feats: jnp.ndarray, in_dim) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.nets.MLP([in_dim, in_dim  ], activation=jax.nn.silu)(feats)
  net = hk.Linear(1)(net) 
  return net


In [31]:
# GAT modified
def GNN(add_self_edges: bool = False ) -> Callable:
  def _ApplyGNN(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
 
    nodes, edges, receivers, senders, _, _, _ = graph
    
    
    # Equivalent to the sum of n_node, but statically known.
    try:
      sum_n_node = nodes.shape[0]
    except IndexError:
      raise IndexError('GAT requires node features')

    
    
    # position of the nodes are the last 3 entries in the features matrix
    distances = jnp.linalg.norm( nodes[:,-3:][senders] - nodes[:,-3:][receivers], axis=-1)
    node_feat = nodes[:,:n_scalar]
    
    input_dim = nodes[:,:n_scalar].shape[-1]
    
    
    concat_feats = jnp.concatenate([ node_feat[senders], node_feat[receivers], distances.reshape((-1,1))**2 ], axis=-1)
    m_ij = f_e(concat_feats, input_dim)
    
    
    # predict edges
    e_ij = f_inf(m_ij, input_dim)
    
    m_i = jraph_utils.segment_sum( e_ij*m_ij, receivers, num_segments=sum_n_node)

    concat_for_hl1_i =  jnp.concatenate([node_feat,  m_i], axis=-1)
    
    
    new_node_feat = node_feat + f_h(concat_for_hl1_i, input_dim) #h^(l+1)_i    f_h is the node update func

     
    position_mlp = f_x(concat_feats, input_dim)
    prefac_xi_xj = (nodes[:,-3:][senders] - nodes[:,-3:][receivers])/ (distances.reshape((-1,1))+1)
    
    sum_xi_xj =  jraph_utils.segment_sum( prefac_xi_xj*position_mlp, receivers, num_segments=sum_n_node)
    #sum_xi_xj = jnp.einsum('ij,ij ->i', prefac_xi_xj, position_mlp ) #f_x
    
    x_Lplus1_i = nodes[:,-3:] + sum_xi_xj
      
    
    

    x_Lplus1_i = x_Lplus1_i/jnp.linalg.norm(x_Lplus1_i, axis=-1 )[..., jnp.newaxis]
    mask = jnp.isnan(x_Lplus1_i)  # boolean mask selecting non-nan values
    x_Lplus1_i = jnp.where(~mask, x_Lplus1_i, 0)
    
    #x_Lplus1_i = jnp.nan_to_num(x_Lplus1_i, copy=False,nan=0.0, posinf=0.0, neginf=0.0)#    x_Lplus1_i.at[jnp.isnan(x_Lplus1_i)].set(0.0)
    
    
    nodes = jnp.concatenate([new_node_feat, nodes[:,n_scalar:-3], x_Lplus1_i],axis=-1)
    #print(nodes.shape)
    return graph._replace(nodes=nodes)

  # pylint: enable=g-long-lambda
  return _ApplyGNN

In [32]:
def dim_expand_MLP(feats: jnp.ndarray, out_dim) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.nets.MLP( [out_dim] )(feats)
  return net


def dim_expand(graph: jraph.GraphsTuple, out_dim) -> jraph.GraphsTuple:

    nodes, edges, receivers, senders, _, _, _ = graph
    node_feat = nodes[:,:n_scalar]
    new_node = dim_expand_MLP(node_feat, out_dim)
    nodes = jnp.concatenate([new_node, nodes[:,n_scalar:]],axis=-1)
    return graph._replace(nodes=nodes)
 

## The Noiser: takes in a batch ads noise and estimates the score

In [33]:
@jax.jit
def noiser(nodes_I, key, noise_dist_std=noise_dist_std):
    key1, key2 =jax.random.split(key)
    # Sample random noise from target noise distribution
    s = noise_dist_std * jnp.abs(jax.random.normal(shape=[nodes_I.shape[0]], key=key1)) + 1e-3
    @jax.vmap
    def sample(quatX, scale,   seed ):
        key3, key4 =jax.random.split(seed)
        
        dist_quat = IsotropicGaussianSO3(quatX, scale)
        quatY = dist_quat.sample(seed=key3)
 
        
        def fn_quat(s_direction, val):
            return dist_quat.log_prob( (SO3(val) @ SO3.exp(s_direction)).wxyz) 
        score_so3 = jax.grad(fn_quat)(jnp.zeros(3), quatY )   #direction of the derivative, can be chosen to be the identity
        
        return quatY, score_so3, scale.reshape([1])
        #return { 'x_so3': quatX, 'y_so3': quatY, 
                 #'score_so3': score_so3, 's':scale.reshape([1]) }
    
    
    # Generates training set batch
  
    return sample( nodes_I, s, jax.random.split(key2, nodes_I.shape[0]))



In [34]:

def MLP_so3(so3, feats, stdev, tensor_T):
    so3_log = jax.vmap(lambda u: SO3(u).log())(so3)
    net = jnp.concatenate([so3_log, feats, stdev, tensor_T],axis=-1)
    net = hk.nets.MLP([256,256,256], activation=jax.nn.leaky_relu)(net)
    net = hk.Linear( 3)(net)  
    return net/stdev 


def so3_SGM_layer(graph: jraph.GraphsTuple, key) -> jraph.GraphsTuple:

    nodes, edges, receivers, senders, _, _, _ = graph
    
    nodes_scalar = nodes[:, :n_scalar]
    nodes_so3 = nodes[:,n_scalar:-3]
    nodes_I = nodes_so3[:,:4]
    nodes_T = nodes_so3[:,4:]
    
    noised_I, noised_I_score, stdev = noiser(nodes_I, key, noise_dist_std=noise_dist_std)
    
    output_score = MLP_so3(noised_I, nodes_scalar, stdev, nodes_T)
    return output_score, noised_I_score
#     nodes = jnp.concatenate([new_node, nodes[:,n_scalar:]],axis=-1)
    
#     return graph._replace(nodes=nodes)

In [35]:
 

def gcn_definition(graph: jraph.GraphsTuple, key) -> jraph.GraphsTuple:
  """Defines a GCN for the karate club task.
  Args:
    graph: GraphsTuple the network processes.

  Returns:
    output graph with updated node values.
  """
  graph = dim_expand(graph,   32)
    
  gn = GNN( 
      add_self_edges=False )
  graph = gn(graph)
  
  

  gn = GNN(add_self_edges=False) # output dim is 2 because we have 2 scalars.
  graph = gn(graph)
    
  graph = dim_expand(graph,   32)
  #key = next(rng)
  output_score, true_score = so3_SGM_layer(graph, key )#only do node operations, no messages


  return output_score, true_score

In [63]:
g_init

GraphsTuple(nodes=array([[ 2.9130838 ,  3.0667794 ,  7.3337674 , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.1524162 ,  1.1580101 ,  6.9149356 , ..., -0.08234406,
        -0.17825317, -0.22185516],
       [ 0.87462974,  0.64877856,  4.3584447 , ..., -0.04729843,
         0.22956848,  0.7696276 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32), edges=array([-1], dtype=int32), receivers=array([ 1,  2,  3, ..., -1, -1, -1], dtype=int32), senders=array([ 0,  0,  0, ..., -1, -1, -1], dtype=int32), globals=array([-1], dtype=int32), n_node=array([16], dtype=int32), n_edge=array([224], dtype=int32))

In [64]:
network = hk.without_apply_rng(hk.transform(gcn_definition ))


#params = jax.vmap(lambda x: network.init( next(rng) , x))  (g)

params =  network.init( next(rng) , g_init, next(rng))

#out_graph = jax.vmap(lambda x,y: network.apply(x,y))   (params, g)
 

Invalid nan value encountered in the output of a C++-jit/pmap function. Calling the de-optimized version.


FloatingPointError: invalid value (nan) encountered in jit(div)

In [61]:
params

{'linear': {'b': Array([nan, nan, nan, nan, nan], dtype=float32),
  'w': Array([[nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan]], dtype=float32)},
 'linear_1': {'b': Array([nan], dtype=float32),
  'w': Array([[nan],
         [nan],
         [nan],
         [nan],
         [nan]], dtype=float32)},
 'linear_2': {'b': Array([nan, nan, nan, nan, nan], dtype=float32),
  'w': Array([[nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan]], dtype=float32)},
 'linear_3': {'b': Array([0.], dtype=float32),
  'w': Array([[ 0.21955472],
         [ 0.20609367],
         [-0.09408663],
         [-0.18978944],
         [-0.26372573]], dtype=float32)},
 'linear_4': {'b': Array([nan, nan, nan], dtype=float32),
  'w': Array([[nan, nan, nan],
         [nan, nan, nan],
    

In [54]:
@jax.jit
def prediction_loss(params: hk.Params, graph_batch: jraph.GraphsTuple, key ) -> jnp.ndarray:
    
    predicted_graph = jax.vmap ( lambda x, y: network.apply(params, x, y)) (graph_batch, key)
    score_pred = predicted_graph[0]
    score_true = predicted_graph[1]
    
    loss =  jnp.linalg.norm(score_pred   -  score_true, axis=-1) **2
    
    return jnp.mean(loss)



In [55]:
opt_init, opt_update = optax.adam(1e-3)
opt_state = opt_init(params)

In [56]:
@jax.jit
def update(params: hk.Params, opt_state, graph_batch: jraph.GraphsTuple, key) -> Tuple[hk.Params, Any]:
    
    loss, grads = jax.value_and_grad(prediction_loss)(params, graph_batch, key)

    updates, opt_state = opt_update(grads, opt_state)
    
    return optax.apply_updates(params, updates), opt_state, loss


In [57]:
losses = []
params_list = []

In [59]:
from jax import config 
config.update("jax_debug_nans", True)

In [60]:
for step in tqdm(range(10_000)):
    params, opt_state, loss = update(params, opt_state, next(dset), jax.random.split(next(rng), 64 )  )

    print(loss)
    if jnp.isnan(loss):
        break
    
    if step%10==0 or step==0:
        pickle.dump( params_list, open( "/ocean/projects/phy210062p/yjagvara/EGNN_so3_weights.p", "wb" ) )
        pickle.dump( losses, open( "/ocean/projects/phy210062p/yjagvara/EGNN_so3_losses.p", "wb" ) )
        losses.append(loss)
        params_list.append(params)

  0%|          | 0/10000 [00:09<?, ?it/s]


FloatingPointError: invalid value (nan) encountered in jit(dot_general)

losses = pickle.load(open( "/ocean/projects/phy210062p/yjagvara/EGNN_losses.p", "rb" ))
params_list = pickle.load(open( "/ocean/projects/phy210062p/yjagvara/EGNN_weights.p", "rb" ))

In [ ]:
plot(losses)

In [ ]:
predicted_graph = jax.vmap ( lambda x: network.apply(params_list[-1], x)) (GP)

In [ ]:
predicted_graph.nodes

In [ ]:
GP.nodes[:,:,n_scalar:n_scalar+3]

In [ ]:
t = jnp.einsum('...k,...k->...' ,  predicted_graph.nodes, GP.nodes[:,:,n_scalar:n_scalar+3])

In [ ]:
t_new = np.ma.masked_equal(t,0)
print(t_new.shape)


In [ ]:
x=hist(t_new[:20],bins=10,density=False)

In [ ]:
a=hist(t_new[5],bins=20,density=True)

In [ ]:

@jraph.concatenated_args
def node_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.Sequential(
      [hk.Linear(128), jax.nn.relu,
       hk.Linear(128)])
  return net(feats)


def net_fn(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
  # Add a global paramater for graph classification.
  graph = graph._replace(globals=jnp.zeros([graph.n_node.shape[0], 1]))
  embedder = jraph.GraphMapFeatures(
      hk.Linear(128), hk.Linear(128), hk.Linear(128))
  net = jraph.GraphNetwork(
      update_node_fn=node_update_fn,
      update_edge_fn=edge_update_fn,
      update_global_fn=update_global_fn)
  return net(embedder(graph)) 

In [ ]:
def compute_loss(params: hk.Params, graph: jraph.GraphsTuple, label: jnp.ndarray,
                 net: jraph.GraphsTuple) -> Tuple[jnp.ndarray, jnp.ndarray]:
  """Computes loss and accuracy."""
  pred_graph = net.apply(params, graph)
  preds = jax.nn.log_softmax(pred_graph.globals)
  targets = jax.nn.one_hot(label, 2)

  # Since we have an extra 'dummy' graph in our batch due to padding, we want
  # to mask out any loss associated with the dummy graph.
  # Since we padded with `pad_with_graphs` we can recover the mask by using
  # get_graph_padding_mask.
  mask = jraph.get_graph_padding_mask(pred_graph)

  # Cross entropy loss.
  loss = -jnp.mean(preds * targets * mask[:, None])

  # Accuracy taking into account the mask.
  accuracy = jnp.sum(
      (jnp.argmax(pred_graph.globals, axis=1) == label) * mask) / jnp.sum(mask)
  return loss, accuracy

In [ ]:
# Adapted from https://github.com/deepmind/jraph/blob/master/jraph/ogb_examples/train.py
def train(dataset: List[Dict[str, Any]], num_train_steps: int) -> hk.Params:
  """Training loop."""

  # Transform impure `net_fn` to pure functions with hk.transform.
  net = hk.without_apply_rng(hk.transform(net_fn))
  # Get a candidate graph and label to initialize the network.
  graph = dataset[0]['input_graph']

  # Initialize the network.
  params = net.init(jax.random.PRNGKey(42), graph)
  # Initialize the optimizer.
  opt_init, opt_update = optax.adam(1e-4)
  opt_state = opt_init(params)

  compute_loss_fn = functools.partial(compute_loss, net=net)
  # We jit the computation of our loss, since this is the main computation.
  # Using jax.jit means that we will use a single accelerator. If you want
  # to use more than 1 accelerator, use jax.pmap. More information can be
  # found in the jax documentation.
  compute_loss_fn = jax.jit(jax.value_and_grad(
      compute_loss_fn, has_aux=True))

  for idx in range(num_train_steps):
    graph = dataset[idx % len(dataset)]['input_graph']
    label = dataset[idx % len(dataset)]['target']
    # Jax will re-jit your graphnet every time a new graph shape is encountered.
    # In the limit, this means a new compilation every training step, which
    # will result in *extremely* slow training. To prevent this, pad each
    # batch of graphs to the nearest power of two. Since jax maintains a cache
    # of compiled programs, the compilation cost is amortized.
    graph = pad_graph_to_nearest_power_of_two(graph)

    # Since padding is implemented with pad_with_graphs, an extra graph has
    # been added to the batch, which means there should be an extra label.
    label = jnp.concatenate([label, jnp.array([0])])

    (loss, acc), grad = compute_loss_fn(params, graph, label)
    updates, opt_state = opt_update(grad, opt_state, params)
    params = optax.apply_updates(params, updates)
    if idx % 50 == 0:
      print(f'step: {idx}, loss: {loss}, acc: {acc}')
  print('Training finished')
  return params

In [ ]:
params = train(train_mutag_ds, num_train_steps=500)

In [ ]:
class MLP(hk.Module):
  def __init__(self, features: jnp.ndarray):
    super().__init__()
    self.features = features

  def __call__(self, x: jnp.ndarray) -> jnp.ndarray:
    layers = []
    for feat in self.features[:-1]:
      layers.append(hk.Linear(feat))
      layers.append(jax.nn.relu)
    layers.append(hk.Linear(self.features[-1]))

    mlp = hk.Sequential(layers)
    return mlp(x)

# Use MLP block to define the update node function
update_node_fn = lambda x: MLP(features=[8, 4])(x)

In [ ]:
# Adapted from https://github.com/deepmind/jraph/blob/master/jraph/_src/models.py#L506
def GraphConvolution(update_node_fn: Callable,
                     aggregate_nodes_fn: Callable = jax.ops.segment_sum,
                     add_self_edges: bool = False,
                     symmetric_normalization: bool = False) -> Callable:
  """Returns a method that applies a Graph Convolution layer.

  Graph Convolutional layer as in https://arxiv.org/abs/1609.02907,
  NOTE: This implementation does not add an activation after aggregation.
  If you are stacking layers, you may want to add an activation between
  each layer.
  Args:
    update_node_fn: function used to update the nodes. In the paper a single
      layer MLP is used.
    aggregate_nodes_fn: function used to aggregates the sender nodes.
    add_self_edges: whether to add self edges to nodes in the graph as in the
      paper definition of GCN. Defaults to False.
    symmetric_normalization: whether to use symmetric normalization. Defaults to
      True.

  Returns:
    A method that applies a Graph Convolution layer.
  """

  def _ApplyGCN(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
    """Applies a Graph Convolution layer."""
    nodes, _, receivers, senders, _, _, _ = graph

    # First pass nodes through the node updater.
    nodes = update_node_fn(nodes)
    # Equivalent to jnp.sum(n_node), but jittable
    total_num_nodes = tree.tree_leaves(nodes)[0].shape[0]
    if add_self_edges:
      # We add self edges to the senders and receivers so that each node
      # includes itself in aggregation.
      # In principle, a `GraphsTuple` should partition by n_edge, but in
      # this case it is not required since a GCN is agnostic to whether
      # the `GraphsTuple` is a batch of graphs or a single large graph.
      conv_receivers, conv_senders = add_self_edges_fn(receivers, senders,
                                                       total_num_nodes)
    else:
      conv_senders = senders
      conv_receivers = receivers

    # pylint: disable=g-long-lambda
    if symmetric_normalization:
      # Calculate the normalization values.
      count_edges = lambda x: jax.ops.segment_sum(
          jnp.ones_like(conv_senders), x, total_num_nodes)
      sender_degree = count_edges(conv_senders)
      receiver_degree = count_edges(conv_receivers)

      # Pre normalize by sqrt sender degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x: x * jax.lax.rsqrt(jnp.maximum(sender_degree, 1.0))[:, None],
          nodes,
      )
      # Aggregate the pre-normalized nodes.
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
      # Post normalize by sqrt receiver degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x:
          (x * jax.lax.rsqrt(jnp.maximum(receiver_degree, 1.0))[:, None]),
          nodes,
      )
    else:
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
    # pylint: enable=g-long-lambda
    return graph._replace(nodes=nodes)

  return _ApplyGCN

In [ ]:
def gcn(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """Defines a graph neural network with 3 GCN layers.
  Args:
    graph: GraphsTuple the network processes.

  Returns:
    output graph with updated node values.
  """
  gn = GraphConvolution(
      update_node_fn=lambda n: jax.nn.relu(hk.Linear(8)(n)),
      add_self_edges=False)
  graph = gn(graph)

  gn = GraphConvolution(
      update_node_fn=lambda n: jax.nn.relu(hk.Linear(4)(n)),
      add_self_edges=False)
  graph = gn(graph)

  gn = GraphConvolution(
      update_node_fn=hk.Linear(2))
  graph = gn(graph)
  return graph

In [ ]:
# Adapted from https://github.com/deepmind/jraph/blob/master/jraph/_src/models.py#L506
def GraphConvolution(update_node_fn: Callable,
                     aggregate_nodes_fn: Callable = jax.ops.segment_sum,
                     add_self_edges: bool = False,
                     symmetric_normalization: bool = False) -> Callable:
  """Returns a method that applies a Graph Convolution layer.

  Graph Convolutional layer as in https://arxiv.org/abs/1609.02907,
  NOTE: This implementation does not add an activation after aggregation.
  If you are stacking layers, you may want to add an activation between
  each layer.
  Args:
    update_node_fn: function used to update the nodes. In the paper a single
      layer MLP is used.
    aggregate_nodes_fn: function used to aggregates the sender nodes.
    add_self_edges: whether to add self edges to nodes in the graph as in the
      paper definition of GCN. Defaults to False.
    symmetric_normalization: whether to use symmetric normalization. Defaults to
      True.

  Returns:
    A method that applies a Graph Convolution layer.
  """

  def _ApplyGCN(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
    """Applies a Graph Convolution layer."""
    nodes, _, receivers, senders, _, _, _ = graph

    # First pass nodes through the node updater.
    nodes = update_node_fn(nodes)
    # Equivalent to jnp.sum(n_node), but jittable
    total_num_nodes = tree.tree_leaves(nodes)[0].shape[0]
    if add_self_edges:
      # We add self edges to the senders and receivers so that each node
      # includes itself in aggregation.
      # In principle, a `GraphsTuple` should partition by n_edge, but in
      # this case it is not required since a GCN is agnostic to whether
      # the `GraphsTuple` is a batch of graphs or a single large graph.
      conv_receivers, conv_senders = add_self_edges_fn(receivers, senders,
                                                       total_num_nodes)
    else:
      conv_senders = senders
      conv_receivers = receivers

    # pylint: disable=g-long-lambda
    if symmetric_normalization:
      # Calculate the normalization values.
      count_edges = lambda x: jax.ops.segment_sum(
          jnp.ones_like(conv_senders), x, total_num_nodes)
      sender_degree = count_edges(conv_senders)
      receiver_degree = count_edges(conv_receivers)

      # Pre normalize by sqrt sender degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x: x * jax.lax.rsqrt(jnp.maximum(sender_degree, 1.0))[:, None],
          nodes,
      )
      # Aggregate the pre-normalized nodes.
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
      # Post normalize by sqrt receiver degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x:
          (x * jax.lax.rsqrt(jnp.maximum(receiver_degree, 1.0))[:, None]),
          nodes,
      )
    else:
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
    # pylint: enable=g-long-lambda
    return graph._replace(nodes=nodes)

  return _ApplyGCN

In [ ]:
# GAT modified
def GNN(add_self_edges: bool = False ) -> Callable:
 
 

  def _ApplyGNN(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
 
    nodes, edges, receivers, senders, _, _, _ = graph
    
    
    #Equivalent to the sum of n_node, but statically known.
    try:
      sum_n_node = nodes.shape[0]
    except IndexError:
      raise IndexError('GAT requires node features')

    
    
    
    
    # position of the nodes are the last 3 entries in the features matrix
    
    distances = jnp.linalg.norm( nodes[:,:3][senders] - nodes[:,:3][receivers], axis=-1)
    node_feat = nodes[:,3:]
    
    input_dim = nodes[:,3:].shape[-1]
    
    
    concat_feats = jnp.concatenate([ node_feat[senders], node_feat[receivers], distances.reshape((-1,1))**2 ], axis=-1)
    m_ij = f_e(concat_feats, input_dim)
    
    
    # predict edges
    e_ij = f_inf(m_ij, input_dim)
    
    m_i = jraph_utils.segment_sum( e_ij*m_ij, receivers, num_segments=sum_n_node)

    concat_for_hl1_i =  jnp.concatenate([node_feat,  m_i], axis=-1)
    
    
    new_node_feat = node_feat + f_h(concat_for_hl1_i, input_dim) #h^(l+1)_i    f_h is the node update func

     
    position_mlp = f_x(concat_feats, input_dim)
    prefac_xi_xj = (nodes[:,:3][senders] - nodes[:,:3][receivers])/ (distances.reshape((-1,1))+1)
    
    sum_xi_xj =  jraph_utils.segment_sum( prefac_xi_xj*position_mlp, receivers, num_segments=sum_n_node)
    #sum_xi_xj = jnp.einsum('ij,ij ->i', prefac_xi_xj, position_mlp ) #f_x
    
    x_Lplus1_i = nodes[:,:3] + sum_xi_xj
    
    nodes = jnp.concatenate([x_Lplus1_i, new_node_feat],axis=-1)
    #print(nodes.shape)
    return graph._replace(nodes=nodes)

  # pylint: enable=g-long-lambda
  return _ApplyGNN

In [ ]:
def dim_expand_MLP(feats: jnp.ndarray, out_dim) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.Linear( out_dim )(feats)
  return net


def dim_expand(graph: jraph.GraphsTuple, out_dim) -> jraph.GraphsTuple:

    nodes, edges, receivers, senders, _, _, _ = graph

    node_feat = nodes[:,3:]
    in_dim = nodes[:,3:].shape[-1]
    new_node = dim_expand_MLP(node_feat, out_dim)
    
    nodes = jnp.concatenate([nodes[:,:3], new_node],axis=-1)

     
    return graph._replace(nodes=nodes)
 